In [85]:
import cv2
import numpy as np
import math
from matplotlib import pyplot as plt
from yolov7.disk_detector import DiskDetector
from extract_metadata import extract_metadata
import pandas as pd
import plotly.express as px
from math import sin, cos, sqrt, atan2, radians
import numpy as np
from pymap3d import vincenty

class DEFAULTS:
    average_lat = 31.9686
    average_alt = 98 # Height from sea level [m]
    debug = True
    show_result = True
    thresh = 10
    noise_atten = 50
    starting_lat = 49.25873130555556
    starting_lon = -123.24252083333333
    true_lat =  49.258777305555554
    true_lon =  -123.24260088888889

In [86]:
def x_algorithm(x1, y1, theta1, x2, y2, theta2):
    '''
    Geometry at the core of the X algorithm, written in a separate function cause I tried to make it run faster but failed lol.
    Takes in the relative cartesian position of the camera from the starting point and the angles between the camera and the disks.
    Returns the relative cartesian position of the landing zone from the starting point, to be converted to GPS coordinates.

    A better breakdown of how this works can be seen in the x.py script.
    '''

    m1 = -np.tan(theta1)
    m2 = -np.tan(theta2)
    if m1 == m2:
        return None, None

    x = ((y2 - m2 * x2) - (y1 - m1 * x1)) / (m1 - m2)
    y = m1 * x + (y1 - m1 * x1)

    if x > 10000 or x < -10000:

        if False:
            print("Found outlier")
            print(f"t1:{theta1} t2:{theta2}")
            print(f"x1:{x1} y1:{y1} x2:{x2} y2:{y2}")
            print(f"disk:{detected_disk.id} disk2: {detected_disk.id}")

        return None, None

    return -x, y

In [87]:
def gps2cartesian(lat, lon, lat_0 = DEFAULTS.starting_lat, lon_0 = DEFAULTS.starting_lon):
        avelat = DEFAULTS.average_lat
        
        # return x, y
        radiusatlocation = math.sqrt((((((6378.137**2)*math.cos(avelat))**2)
                           + ((6356.752**2)*math.sin(avelat))**2))
                           /(((6378.137*math.cos(avelat))**2)
                           + ((6356.752*math.sin(avelat))**2)))

        # print('Radius in m at location =', radiusatlocation)

        x = radiusatlocation * (lon - lon_0)*(math.pi/180) * 1000
        y = radiusatlocation * (lat - lat_0)*(math.pi/180) * 1000

        return x, y
    
def cartesian2gps(x, y, lat_0 = DEFAULTS.starting_lat, lon_0 = DEFAULTS.starting_lon):

    # just the opposite of gps2cartesian
    avelat = DEFAULTS.average_lat

    radiusatlocation = math.sqrt((((((6378.137**2)*math.cos(avelat))**2)
                        + ((6356.752**2)*math.sin(avelat))**2))
                        /(((6378.137*math.cos(avelat))**2)
                        + ((6356.752*math.sin(avelat))**2)))

    # print('Radius in m at location =', radiusatlocation)

    lon = x / (radiusatlocation*(math.pi/180) * 1000) + lon_0
    lat = y / (radiusatlocation*(math.pi/180) * 1000) + lat_0

    return lat, lon
    

In [88]:
disk_detector = DiskDetector()
disk_detector.setup()

YOLOR  d5dabb4 torch 2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)



Fusing layers... 


Model Summary: 314 layers, 36481772 parameters, 6194944 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


In [89]:
def analyze(disk_detector, filepath):
    
    image = cv2.imread(filepath)
    metadata = extract_metadata(filepath)
    
    lat = metadata['lat']
    lon = metadata['lon']
    
    x, y = gps2cartesian(lat, lon)
    print(x, y)
    
    
    h, w, _ = image.shape
    for cX, cY in disk_detector.predict(image):
        if cX is None:
            break

        # line between disk and center of frame
        v1 = np.array([cX-w/2, cY-h/2])
        v2 = np.array([1, 0])
        dot = np.dot(v1, v2)
        det = v1[0]*v2[1] - v1[1]*v2[0]

        # angle between disk and center of frame
        #angle = np.arctan2(cY - h/2, cX - w/2)
        angle = math.atan2(det, dot)

        # maybe this is hacky but it works
        if angle < 0:
            angle += 2*np.pi

    return x, y, lat, lon, angle

In [90]:
x1, y1, lat1, lon1, theta1 = analyze(disk_detector, "./data/start.JPG")
x2, y2, lat2, lon2, theta2 = analyze(disk_detector, "./data/stop.JPG" )


0.0 0.0
-21.230138267320772 2.9966871532815516


In [91]:
x_predicted, y_predicted = x_algorithm(x1, y1, theta1, x2, y2, theta2)
lat_predicted, lon_predicted = cartesian2gps(x_predicted, y_predicted)

In [92]:
import plotly.graph_objects as go

df = pd.DataFrame({
    'Latitude': [lat_predicted, DEFAULTS.true_lat],
    'Longitude': [lon_predicted, DEFAULTS.true_lon],
    'Label': ['Prediction', 'Truth']
})

# Create scatter plot
fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude', text='Label')

# Add connecting line
fig.add_trace(
    go.Scattermapbox(
        lat=df['Latitude'],
        lon=df['Longitude'],
        mode='markers+text',
        text=df['Label'],
        textposition="top right",
        marker=dict(size=10)
    )
)

# Update layout with Mapbox properties
fig.update_layout(mapbox_style="open-street-map")


# Show the plot

import plotly.offline as pyo
pyo.plot(fig, filename='prediction_vs_truth.html', auto_open=True)

'prediction_vs_truth.html'

In [93]:
def simpleHav(lat1, long1, lat2, long2, Bearing):
    """
    Given 2 positions provide the distance (shortest distance) great circle arc.
    Inputs in degrees lat long
    Output is a length in metres
    """
    
    AverageR = 6371000  # Earth Radius

    a = 6378137 #Semi Major Axis a
    b = 6356752 #Semi Minor Axis b
    e = np.sqrt(1-(b**2/a**2)) #eccentricity
    
    rlat1  = np.radians(lat1)
    rlong1 = np.radians(long1)
    rlat2  = np.radians(lat2)
    rlong2 = np.radians(long2)
    rBearing = np.radians(Bearing)

    GEOcentricRadius = np.sqrt(((a**2*np.cos(rlat1))**2 + (b**2*np.sin(rlat1))**2)/((a*np.cos(rlat1))**2 + (b*np.sin(rlat1))**2))        
    
    RN = a/np.sqrt(1-e**2*np.sin(rlat1)**2)         #Radius of Curvature in Prime Vertical, terminated by minor axis
    RM = RN * ((1-e**2)/(1-e**2*np.sin(rlat1)**2))  #Radius of Curvature: in Meridian 
    RadiusofCurvature = 1/(np.cos(rBearing)**2/RM + np.sin(rBearing)**2/RN) #Radius of Curvature at azimuth


    arclength = np.arccos(np.sin(rlat1)*np.sin(rlat2) + np.cos(rlat1)*np.cos(rlat2)*np.cos(rlong2-rlong1)  )
        
    distance  = arclength * AverageR
    distance1 = arclength * GEOcentricRadius
    distance2 = arclength * RadiusofCurvature
    
    return distance, distance1, distance2

print("Distance:", haversine((lat_predicted, lon_predicted), (DEFAULTS.true_lat, DEFAULTS.true_lon)), "km")







Distance: 0.021163647447305298 km
